## **VacationPy**

Using the data retrieved in the WeatherPy exercise, I narrowed down the data to my ideal weather conditions:

* Temperature between 65F and 80F
* Humidity between 30 and 50
* Cloudiness at less than 20%
* Wind Speed at less than 15mph

Using the Google Places API, I found a hotel within a 5000 meter radius for each city on my list. After adding the hotel to the list, I created a heatmap using gmaps with markers noting the hotel that was chosen for each city on my list.

### Dependencies and Setup

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part 1 Results into DataFrame

In [2]:
# Pull in the weather data from WeatherPy
weather_path = ("..\WeatherPy\data\clean_city_data.csv")

# Read in the data and display data table
weather_data = pd.read_csv(weather_path)
weather_data.head()

,City Name,Latitude,Longitude,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lebu,-37.62,-73.65,51.57,51.57,72,61,11.32,CL,1596659003
1,Torgau,51.57,13.00,66.00,69.80,43,40,5.82,DE,1596659003
2,Algiers,36.75,3.04,77.00,77.00,65,0,13.87,DZ,1596658984
3,Oistins,13.07,-59.53,87.80,87.80,62,40,16.11,BB,1596659004
4,Vila Franca do Campo,37.72,-25.43,73.40,73.40,83,40,10.29,PT,1596658999


### Humidity Heatmap

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Create variables for locations and humidityz
locations = weather_data[["Latitude", "Longitude"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [5]:
# Create humidity headmap
fig = gmaps.figure(zoom_level=1.5,center=(30,30))
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create DataFrame with ideal weather conditions

In [6]:
# Filter for ideal weather conditions
ideal_temp = (weather_data["Min Temp"]>=65) & (weather_data["Max Temp"]<80)
ideal_humidity = (weather_data["Humidity"]>=30) & (weather_data["Humidity"]<50)
ideal_cloud = (weather_data["Cloudiness"]< 20)
ideal_wind = (weather_data["Wind Speed"]< 15)

#Create DataFrame with these weather conditions
ideal_temp_df = (weather_data[ideal_temp & ideal_humidity & ideal_cloud & ideal_wind]).reset_index(drop=True)
ideal_temp_df

,City Name,Latitude,Longitude,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mayskiy,47.69,40.10,78.80,78.80,44,0,13.42,RU,1596659008
1,Trélazé,47.45,-0.47,69.80,75.00,46,3,8.01,FR,1596659015
2,Mandalgovi,45.76,106.27,69.21,69.21,37,0,11.21,MN,1596659062
3,Oliveira,-20.70,-44.83,68.14,68.14,41,0,7.40,BR,1596659097
4,Erenhot,43.65,111.98,73.62,73.62,45,0,12.93,CN,1596659106
5,Atikokan,48.76,-91.62,79.75,79.75,30,3,6.53,CA,1596659114
6,Zhezkazgan,47.80,67.71,79.90,79.90,31,18,11.50,KZ,1596659124
7,San Juan,-31.54,-68.54,72.01,72.01,31,0,7.52,AR,1596658896
8,San Vicente,-26.62,-54.13,74.03,74.03,49,0,2.68,AR,1596659157


### Hotel Map

In [7]:
# Create a new DataFrame called hotel_df
hotel_df = ideal_temp_df

# Add an empty column for Hotel Name
    # www.interviewqs.com/ddi_code_snippets/add_new_col_df_default_value

hotel_df.insert(1, "Hotel Name", "")
hotel_df

,City Name,Hotel Name,Latitude,Longitude,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mayskiy,,47.69,40.10,78.80,78.80,44,0,13.42,RU,1596659008
1,Trélazé,,47.45,-0.47,69.80,75.00,46,3,8.01,FR,1596659015
2,Mandalgovi,,45.76,106.27,69.21,69.21,37,0,11.21,MN,1596659062
3,Oliveira,,-20.70,-44.83,68.14,68.14,41,0,7.40,BR,1596659097
4,Erenhot,,43.65,111.98,73.62,73.62,45,0,12.93,CN,1596659106
5,Atikokan,,48.76,-91.62,79.75,79.75,30,3,6.53,CA,1596659114
6,Zhezkazgan,,47.80,67.71,79.90,79.90,31,18,11.50,KZ,1596659124
7,San Juan,,-31.54,-68.54,72.01,72.01,31,0,7.52,AR,1596658896
8,San Vicente,,-26.62,-54.13,74.03,74.03,49,0,2.68,AR,1596659157


In [8]:
# Set parameters to search for hotels within 5000 meters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Set the base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through each row in the DataFrame to get the lat/long and add to parameters
    # stackoverflow.com/questions/1024847/how-can-i-add-new-keys-to-a-dictionary
    
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]

    params["location"] = f"{lat},{lng}"

# Run the request using our params dictionary
    response = requests.get(base_url, params=params)

# Convert response to json
    hotel_data = response.json()
    
# Save hotel name to DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    except Exception:
        pass
        

In [9]:
# Drop any cities with blank results
    # www.kite.com/python/answers/how-to-drop-empty-rows-from-a-pandas-dataframe-in-python

nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df = hotel_df.reset_index(drop=True)

# Show updated DataFrame
hotel_df

,City Name,Hotel Name,Latitude,Longitude,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mayskiy,Dio Lakrua,47.69,40.10,78.80,78.80,44,0,13.42,RU,1596659008
1,Trélazé,Logis Hôtel de Loire Rest. les Bateliers,47.45,-0.47,69.80,75.00,46,3,8.01,FR,1596659015
2,Mandalgovi,В.Ганбат,45.76,106.27,69.21,69.21,37,0,11.21,MN,1596659062
3,Oliveira,Hotel Vila Mineira | Oliveira MG,-20.70,-44.83,68.14,68.14,41,0,7.40,BR,1596659097
4,Erenhot,Erenhot Rurouni Youth Hostel,43.65,111.98,73.62,73.62,45,0,12.93,CN,1596659106
5,Atikokan,Camp Quetico,48.76,-91.62,79.75,79.75,30,3,6.53,CA,1596659114
6,Zhezkazgan,Kvartiry Posutochno.,47.80,67.71,79.90,79.90,31,18,11.50,KZ,1596659124
7,San Juan,Hotel Provincial,-31.54,-68.54,72.01,72.01,31,0,7.52,AR,1596658896
8,San Vicente,Complejo Turístico Farelluk,-26.62,-54.13,74.03,74.03,49,0,2.68,AR,1596659157


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer on top of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))